In [24]:
from imports import * # Catchall for commonly used imports. Will still throw linter warnings for imports not explicitly called below

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
from bs4 import BeautifulSoup
import requests
import os
import re


import warnings
warnings.filterwarnings('ignore')

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 100)

### Acquire and process Central Alberta cattle auction data from three sites:
- vjvauction.com (Ponoka, Rimbey, and Westlock markets)
- oldsauction.com
- drylandcattle.com (Veteran market)

In [25]:
# Establish urls

vjv_ponoka_url = 'https://vjvauction.com/ponoka/market-report/cattle'
vjv_rimbey_url = 'https://vjvauction.com/rimbey/market-report/cattle'
vjv_westlock_url = 'https://vjvauction.com/westlock/market-report/cattle'
olds_url = 'https://www.oldsauction.com/market-report'
dryland_url = 'http://www.drylandcattle.com/marketreports.aspx'



In [ ]:
# Read tables directly from site
okie_panokie = pd.read_html(vjv_ponoka_url)
okie_panokie
# Extract df from list generated by reading in table
ponoka_df = okie_panokie[0]
ponoka_df.info()
ponoka_df
# Stock and feeder cattle data is of interest to client, so slice down to it
ponoka_stock = ponoka_df.copy().iloc[16:25]
ponoka_stock

# Transpose df to position named rows as columns
#ponoka_df = ponoka_df.T
#ponoka_df

ponoka_stock = ponoka_stock.T
ponoka_stock
# Isolate columns from row
ponoka_stock.iloc[0]
# Set column names to first row
ponoka_stock.columns = ponoka_stock.iloc[0]
# Rename columns to Python conventional name schema
ponoka_stock.columns = [col.lower()\
    .replace(' | ', '_')\
        .replace(' ', '_')\
            .replace('1000', 'thousand')\
                .replace('900', 'nine_hdrd')\
                    .replace('800', 'eight_hdrd')\
                        .replace('700', 'seven_hdrd')\
                            .replace('600', 'six_hdrd')\
                                .replace('500', 'five_hdrd')\
                                    .replace('400', 'four_hdrd')\
                                        .replace('300', 'three_hdrd') for col in ponoka_stock.columns]

ponoka_stock = ponoka_stock.rename(columns = {'weight' : 'sex'})
ponoka_stock = ponoka_stock[1:]

ponoka_stock
print(type(ponoka_stock.index))

flat_ps_idx = ponoka_stock.index.to_flat_index()
print(type(flat_ps_idx))
flat_ps_idx
ponoka_stock1 = ponoka_stock.copy().set_index(flat_ps_idx)
ponoka_stock1
ponoka_stock1['thousand_lbs_range'] = ponoka_stock1.thousand_lbs_steers_heifers.str.replace('$', '').str.replace(' -', ',')

ponoka_stock1.thousand_lbs_range = ponoka_stock1.thousand_lbs_range.str.split(',')



# Make new df to start splitting range list into columns
ponoka_stock1_prices = pd.DataFrame(ponoka_stock1[['sex', 'thousand_lbs_range']])
print(ponoka_stock1_prices)
# Split range column into low and high columns
ponoka_stock1_prices[['thousand_low', 'thousand_high']] = pd.DataFrame(ponoka_stock1_prices.thousand_lbs_range.tolist(), index= ponoka_stock1_prices.index) 
ponoka_stock1_prices.info()
# Convert low and high columns to numeric dtype
ponoka_stock1_prices.thousand_low = ponoka_stock1_prices.thousand_low.astype('float')
ponoka_stock1_prices.thousand_high = ponoka_stock1_prices.thousand_high.astype('float')
# Take average of lows and highs
ponoka_stock1_prices['thousand_avg'] = (ponoka_stock1_prices.thousand_low + ponoka_stock1_prices.thousand_high)/2
ponoka_stock1_prices

In [30]:
ponoka_stock1[['thousand_low', 'thousand_high']] = ponoka_stock1.thousand_lbs_range.to_list()
ponoka_stock1

,sex,thousand_lbs_steers_heifers,nine_hdrd_lbs_steers_heifers,eight_hdrd_lbs_steers_heifers,seven_hdrd_lbs_steers_heifers,six_hdrd_lbs_steers_heifers,five_hdrd_lbs_steers_heifers,four_hdrd_lbs_steers_heifers,three_hdrd_lbs_steers_heifers,thousand_lbs_range,thousand_low,thousand_high
"(Last Auction, Wed Sep 28, 2022)",Steers,$210.00 - $225.00,$225.00 - $238.25,$230.00 - $245.50,$235.00 - $257.00,$245.00 - $266.00,$250.00 - $286.00,$255.00 - $300.00,$255.00 - $318.00,"[210.00, 225.00]",210.00,225.00
"(Last Auction, Wed Sep 28, 2022.1)",Heifers,$180.00 - $208.75,$190.00 - $214.75,$200.00 - $232.50,$215.00 - $236.00,$220.00 - $237.00,$230.00 - $252.00,$240.00 - $269.00,$245.00 - $285.00,"[180.00, 208.75]",180.00,208.75
"(Previous Auction 1, Wed Sep 21, 2022)",Steers,$215.00 - $225.75,$220.00 - $234.25,$230.00 - $251.25,$235.00 - $255.00,$245.00 - $268.00,$255.00 - $289.50,$275.00 - $329.00,$275.00 - $330.00,"[215.00, 225.75]",215.00,225.75
"(Previous Auction 1, Wed Sep 21, 2022.1)",Heifers,$165.00 - $204.00,$210.00 - $226.75,$215.00 - $235.00,$220.00 - $240.00,$220.00 - $244.00,$225.00 - $245.00,$235.00 - $273.00,$245.00 - $291.00,"[165.00, 204.00]",165.00,204.00
"(Previous Auction 2, Wed Sep 14, 2022)",Steers,$210.00 - $222.75,$220.00 - $232.25,$230.00 - $247.00,$240.00 - $260.50,$250.00 - $278.00,$275.00 - $307.00,$275.00 - $305.00,$250.00 - $310.00,"[210.00, 222.75]",210.00,222.75
"(Previous Auction 2, Wed Sep 14, 2022.1)",Heifers,$160.00 - $200.00,$205.00 - $219.00,$210.00 - $230.00,$220.00 - $244.00,$225.00 - $245.00,$230.00 - $250.00,$240.00 - $279.00,$245.00 - $280.00,"[160.00, 200.00]",160.00,200.00


In [26]:
def get_vjv_data(url):
    '''
    Takes in the url from the Vold Jones Vold auction website's market report,
    returns a df produced from the site's tabularized data.
    '''
    # Read table directly from url
    df = pd.read_html(url)
    # Select list item as df
    df = df[0]
    return df

In [27]:
rimbey_table = get_vjv_data(vjv_rimbey_url)
rimbey_table

Click in table to get category history.             Last Auction  \
                              Week Ending:         Tue Sep 20, 2022   
0                               Head Sold:                      732   
1                         Slaughter Cattle         Slaughter Cattle   
2                             D1 - D2 Cows         $90.00 - $107.50   
3                             D3 - D4 Cows          $75.00 - $90.00   
4                       Good Holstein Cows                    — - —   
5                     Medium Holstein Cows                    — - —   
6                              Heiferettes                    — - —   
7                            Bologna Bulls        $115.00 - $139.50   
8                             Feeder Bulls                    — - —   
9                       Replacement Cattle       Replacement Cattle   
10                          Good Bred Cows                    — - —   
11                         Older Bred Cows                    — - —   
12                       Good Bred Heifers                    — - —   
13              Cow / Calf Pairs (Younger)    $1,950.00 - $2,200.00   
14                Cow / Calf Pairs (Older)                    — - —   
15                 Stock and Feeder Cattle  Stock and Feeder Cattle   
16                                  Weight                   Steers   
17               1000 lbs Steers | Heifers        $203.00 - $214.00   
18                900 lbs Steers | Heifers        $206.00 - $226.00   
19                800 lbs Steers | Heifers        $215.00 - $238.50   
20                700 lbs Steers | Heifers        $220.00 - $256.00   
21                600 lbs Steers | Heifers        $230.00 - $266.00   
22                500 lbs Steers | Heifers        $260.00 - $285.00   
23                400 lbs Steers | Heifers        $240.00 - $290.00   
24                300 lbs Steers | Heifers        $260.00 - $289.00   
25                          Dairy & Calves           Dairy & Calves   
26                            Dairy Steers                    — - —   
27                Baby Calves (Dairy) $/Hd                    — - —   
28                 Baby Calves (Beef) $/Hd                    — - —   
29                                    Feed                     Feed   
30                       Hay: Square Bales                    — - —   
31                        Hay: Round Bales                    — - —   
32                     Straw: Square Bales                    — - —   
33                      Straw: Round Bales                    — - —   
34                 Greenfeed: Square Bales                    — - —   
35                  Greenfeed: Round Bales                    — - —   

                                  Previous Auction 1                           \
         Tue Sep 20, 2022.1         Tue Sep 13, 2022       Tue Sep 13, 2022.1   
0                       732                      693                      693   
1          Slaughter Cattle         Slaughter Cattle         Slaughter Cattle   
2          $90.00 - $107.50        $100.00 - $114.00        $100.00 - $114.00   
3           $75.00 - $90.00          $75.00 - $95.00          $75.00 - $95.00   
4                     — - —                    — - —                    — - —   
5                     — - —                    — - —                    — - —   
6                     — - —        $110.00 - $127.00        $110.00 - $127.00   
7         $115.00 - $139.50        $110.00 - $137.00        $110.00 - $137.00   
8                     — - —                    — - —                    — - —   
9        Replacement Cattle       Replacement Cattle       Replacement Cattle   
10                    — - —                    — - —                    — - —   
11                    — - —                    — - —                    — - —   
12                    — - —                    — - —                    — - —   
13    $1,950.00 - $2,200.00                    — - —                    — - —   
14      

In [28]:
westlock_table = get_vjv_data(vjv_westlock_url)
westlock_table

Click in table to get category history.             Last Auction  \
                              Week Ending:         Thu Sep 29, 2022   
0                               Head Sold:                     1941   
1                         Slaughter Cattle         Slaughter Cattle   
2                             D1 - D2 Cows         $96.00 - $119.00   
3                             D3 - D4 Cows          $82.00 - $94.00   
4                       Good Holstein Cows          $85.00 - $91.00   
5                     Medium Holstein Cows          $74.00 - $83.00   
6                              Heiferettes        $113.00 - $170.00   
7                            Bologna Bulls        $120.00 - $151.00   
8                             Feeder Bulls        $130.00 - $159.00   
9                       Replacement Cattle       Replacement Cattle   
10                          Good Bred Cows                    — - —   
11                         Older Bred Cows                    — - —   
12                       Good Bred Heifers                    — - —   
13              Cow / Calf Pairs (Younger)                    — - —   
14                Cow / Calf Pairs (Older)                    — - —   
15                 Stock and Feeder Cattle  Stock and Feeder Cattle   
16                                  Weight                   Steers   
17               1000 lbs Steers | Heifers                    — - —   
18                900 lbs Steers | Heifers        $211.00 - $229.00   
19                800 lbs Steers | Heifers        $218.00 - $230.00   
20                700 lbs Steers | Heifers        $238.00 - $260.00   
21                600 lbs Steers | Heifers        $247.00 - $265.00   
22                500 lbs Steers | Heifers        $260.00 - $294.00   
23                400 lbs Steers | Heifers        $265.00 - $303.00   
24                300 lbs Steers | Heifers        $290.00 - $377.50   
25                          Dairy & Calves           Dairy & Calves   
26                            Dairy Steers                    — - —   
27                Baby Calves (Dairy) $/Hd                    — - —   
28                 Baby Calves (Beef) $/Hd                    — - —   
29                                    Feed                     Feed   
30                       Hay: Square Bales                    — - —   
31                        Hay: Round Bales                    — - —   
32                     Straw: Square Bales                    — - —   
33                      Straw: Round Bales                    — - —   
34                 Greenfeed: Square Bales                    — - —   
35                  Greenfeed: Round Bales                    — - —   

                                  Previous Auction 1                           \
         Thu Sep 29, 2022.1         Thu Sep 22, 2022       Thu Sep 22, 2022.1   
0                      1941                     2270                     2270   
1          Slaughter Cattle         Slaughter Cattle         Slaughter Cattle   
2          $96.00 - $119.00         $95.00 - $114.00         $95.00 - $114.00   
3           $82.00 - $94.00          $80.00 - $93.00          $80.00 - $93.00   
4           $85.00 - $91.00          $80.00 - $91.00          $80.00 - $91.00   
5           $74.00 - $83.00          $69.00 - $79.00          $69.00 - $79.00   
6         $113.00 - $170.00        $118.00 - $161.00        $118.00 - $161.00   
7         $120.00 - $151.00        $115.00 - $148.00        $115.00 - $148.00   
8         $130.00 - $159.00        $140.00 - $170.00        $140.00 - $170.00   
9        Replacement Cattle       Replacement Cattle       Replacement Cattle   
10                    — - —                    — - —                    — - —   
11                    — - —                    — - —                    — - —   
12                    — - —                    — - —                    — - —   
13                    — - —                    — - —                    — - —   
14      

In [32]:
def clean_vjv_table(df):
    '''
    Takes in a df of Vold Jones Vold market report data,
    returns desired slice of df with columns prepared for analysis.
    '''
    # Stock and feeder cattle data is of interest to client, so slice down to it
    df = df.iloc[16:25]
    # Transpose df to position named rows as columns
    df = df.T
    # Isolate columns from row
    df.iloc[0]
    # Set column names to first row
    df.columns = df.iloc[0]
    # Rename columns to Python conventional name schema
    df.columns = [col.lower()\
        .replace(' | ', '_')\
            .replace(' ', '_')\
                .replace('1000', 'thousand')\
                    .replace('900', 'nine_hdrd')\
                        .replace('800', 'eight_hdrd')\
                            .replace('700', 'seven_hdrd')\
                                .replace('600', 'six_hdrd')\
                                    .replace('500', 'five_hdrd')\
                                        .replace('400', 'four_hdrd')\
                                            .replace('300', 'three_hdrd') for col in df.columns]

    # Rename "weight" column to represent its data
    df = df.rename(columns = {'weight' : 'sex'})
    # Drop unneeded first row that supplied column names
    df = df[1:]
    # Flatten MultiIndex
    flat_idx = df.index.to_flat_index()
    df = df.set_index(flat_idx)
    # Create columns of price data converted from str dtype
    df['thousand_lbs_range'] = df.thousand_lbs_steers_heifers.str.replace('$', '').str.replace(' -', ',').str.split(',')
    df[['thousand_low', 'thousand_high']] = df.thousand_lbs_range.to_list()
    df.thousand_low = df.thousand_low.astype('float')
    df.thousand_high = df.thousand_high.astype('float')
    df['thousand_avg'] = (df.thousand_low + df.thousand_high)/2
    df['nine_hdrd_lbs_range'] = df.nine_hdrd_lbs_steers_heifers.str.replace('$', '').str.replace(' -', ',').str.split(',')
    df[['nine_hdrd_low', 'nine_hdrd_high']] = df.nine_hdrd_lbs_range.to_list()
    df.nine_hdrd_low = df.nine_hdrd_low.astype('float')
    df.nine_hdrd_high = df.nine_hdrd_high.astype('float')
    df['nine_hdrd_avg'] = (df.nine_hdrd_low + df.nine_hdrd_high)/2
    df['eight_hdrd_lbs_range'] = df.eight_hdrd_lbs_steers_heifers.str.replace('$', '').str.replace(' -', ',').str.split(',')
    df[['eight_hdrd_low', 'eight_hdrd_high']] = df.eight_hdrd_lbs_range.to_list()
    df.eight_hdrd_low = df.eight_hdrd_low.astype('float')
    df.eight_hdrd_high = df.eight_hdrd_high.astype('float')
    df['eight_hdrd_avg'] = (df.eight_hdrd_low + df.eight_hdrd_high)/2
    df['seven_hdrd_lbs_range'] = df.seven_hdrd_lbs_steers_heifers.str.replace('$', '').str.replace(' -', ',').str.split(',')
    df[['seven_hdrd_low', 'seven_hdrd_high']] = df.seven_hdrd_lbs_range.to_list()
    df.seven_hdrd_low = df.seven_hdrd_low.astype('float')
    df.seven_hdrd_high = df.seven_hdrd_high.astype('float')
    df['seven_hdrd_avg'] = (df.seven_hdrd_low + df.seven_hdrd_high)/2
    df['six_hdrd_lbs_range'] = df.six_hdrd_lbs_steers_heifers.str.replace('$', '').str.replace(' -', ',').str.split(',')
    df[['six_hdrd_low', 'six_hdrd_high']] = df.six_hdrd_lbs_range.to_list()
    df.six_hdrd_low = df.six_hdrd_low.astype('float')
    df.six_hdrd_high = df.six_hdrd_high.astype('float')
    df['six_hdrd_avg'] = (df.six_hdrd_low + df.six_hdrd_high)/2
    df['five_hdrd_lbs_range'] = df.five_hdrd_lbs_steers_heifers.str.replace('$', '').str.replace(' -', ',').str.split(',')
    df[['five_hdrd_low', 'five_hdrd_high']] = df.five_hdrd_lbs_range.to_list()
    df.five_hdrd_low = df.five_hdrd_low.astype('float')
    df.five_hdrd_high = df.five_hdrd_high.astype('float')
    df['five_hdrd_avg'] = (df.five_hdrd_low + df.five_hdrd_high)/2
    df['four_hdrd_lbs_range'] = df.four_hdrd_lbs_steers_heifers.str.replace('$', '').str.replace(' -', ',').str.split(',')
    df[['four_hdrd_low', 'four_hdrd_high']] = df.four_hdrd_lbs_range.to_list()
    df.four_hdrd_low = df.four_hdrd_low.astype('float')
    df.four_hdrd_high = df.four_hdrd_high.astype('float')
    df['four_hdrd_avg'] = (df.four_hdrd_low + df.four_hdrd_high)/2
    df['three_hdrd_lbs_range'] = df.three_hdrd_lbs_steers_heifers.str.replace('$', '').str.replace(' -', ',').str.split(',')
    df[['three_hdrd_low', 'three_hdrd_high']] = df.three_hdrd_lbs_range.to_list()
    df.three_hdrd_low = df.three_hdrd_low.astype('float')
    df.three_hdrd_high = df.three_hdrd_high.astype('float')
    df['three_hdrd_avg'] = (df.three_hdrd_low + df.three_hdrd_high)/2

    return df
    

In [33]:
ponoka_df = get_vjv_data(vjv_ponoka_url)
ponoka_df = clean_vjv_table(ponoka_df)
ponoka_df

,sex,thousand_lbs_steers_heifers,nine_hdrd_lbs_steers_heifers,eight_hdrd_lbs_steers_heifers,seven_hdrd_lbs_steers_heifers,six_hdrd_lbs_steers_heifers,five_hdrd_lbs_steers_heifers,four_hdrd_lbs_steers_heifers,three_hdrd_lbs_steers_heifers,thousand_lbs_range,thousand_low,thousand_high,thousand_avg,nine_hdrd_lbs_range,nine_hdrd_low,nine_hdrd_high,nine_hdrd_avg,eight_hdrd_lbs_range,eight_hdrd_low,eight_hdrd_high,eight_hdrd_avg,seven_hdrd_lbs_range,seven_hdrd_low,seven_hdrd_high,seven_hdrd_avg,six_hdrd_lbs_range,six_hdrd_low,six_hdrd_high,six_hdrd_avg,five_hdrd_lbs_range,five_hdrd_low,five_hdrd_high,five_hdrd_avg,four_hdrd_lbs_range,four_hdrd_low,four_hdrd_high,four_hdrd_avg,three_hdrd_lbs_range,three_hdrd_low,three_hdrd_high,three_hdrd_avg
"(Last Auction, Wed Sep 28, 2022)",Steers,$210.00 - $225.00,$225.00 - $238.25,$230.00 - $245.50,$235.00 - $257.00,$245.00 - $266.00,$250.00 - $286.00,$255.00 - $300.00,$255.00 - $318.00,"[210.00, 225.00]",210.0,225.00,217.500,"[225.00, 238.25]",225.0,238.25,231.625,"[230.00, 245.50]",230.0,245.50,237.750,"[235.00, 257.00]",235.0,257.0,246.00,"[245.00, 266.00]",245.0,266.0,255.5,"[250.00, 286.00]",250.0,286.0,268.00,"[255.00, 300.00]",255.0,300.0,277.5,"[255.00, 318.00]",255.0,318.0,286.5
"(Last Auction, Wed Sep 28, 2022.1)",Heifers,$180.00 - $208.75,$190.00 - $214.75,$200.00 - $232.50,$215.00 - $236.00,$220.00 - $237.00,$230.00 - $252.00,$240.00 - $269.00,$245.00 - $285.00,"[180.00, 208.75]",180.0,208.75,194.375,"[190.00, 214.75]",190.0,214.75,202.375,"[200.00, 232.50]",200.0,232.50,216.250,"[215.00, 236.00]",215.0,236.0,225.50,"[220.00, 237.00]",220.0,237.0,228.5,"[230.00, 252.00]",230.0,252.0,241.00,"[240.00, 269.00]",240.0,269.0,254.5,"[245.00, 285.00]",245.0,285.0,265.0
"(Previous Auction 1, Wed Sep 21, 2022)",Steers,$215.00 - $225.75,$220.00 - $234.25,$230.00 - $251.25,$235.00 - $255.00,$245.00 - $268.00,$255.00 - $289.50,$275.00 - $329.00,$275.00 - $330.00,"[215.00, 225.75]",215.0,225.75,220.375,"[220.00, 234.25]",220.0,234.25,227.125,"[230.00, 251.25]",230.0,251.25,240.625,"[235.00, 255.00]",235.0,255.0,245.00,"[245.00, 268.00]",245.0,268.0,256.5,"[255.00, 289.50]",255.0,289.5,272.25,"[275.00, 329.00]",275.0,329.0,302.0,"[275.00, 330.00]",275.0,330.0,302.5
"(Previous Auction 1, Wed Sep 21, 2022.1)",Heifers,$165.00 - $204.00,$210.00 - $226.75,$215.00 - $235.00,$220.00 - $240.00,$220.00 - $244.00,$225.00 - $245.00,$235.00 - $273.00,$245.00 - $291.00,"[165.00, 204.00]",165.0,204.00,184.500,"[210.00, 226.75]",210.0,226.75,218.375,"[215.00, 235.00]",215.0,235.00,225.000,"[220.00, 240.00]",220.0,240.0,230.00,"[220.00, 244.00]",220.0,244.0,232.0,"[225.00, 245.00]",225.0,245.0,235.00,"[235.00, 273.00]",235.0,273.0,254.0,"[245.00, 291.00]",245.0,291.0,268.0
"(Previous Auction 2, Wed Sep 14, 2022)",Steers,$210.00 - $222.75,$220.00 - $232.25,$230.00 - $247.00,$240.00 - $260.50,$250.00 - $278.00,$275.00 - $307.00,$275.00 - $305.00,$250.00 - $310.00,"[210.00, 222.75]",210.0,222.75,216.375,"[220.00, 232.25]",220.0,232.25,226.125,"[230.00, 247.00]",230.0,247.00,238.500,"[240.00, 260.50]",240.0,260.5,250.25,"[250.00, 278.00]",250.0,278.0,264.0,"[275.00, 307.00]",275.0,307.0,291.00,"[275.00, 305.00]",275.0,305.0,290.0,"[250.00, 310.00]",250.0,310.0,280.0
"(Previous Auction 2, Wed Sep 14, 2022.1)",Heifers,$160.00 - $200.00,$205.00 - $219.00,$210.00 - $230.00,$220.00 - $244.00,$225.00 - $245.00,$230.00 - $250.00,$240.00 - $279.00,$245.00 - $280.00,"[160.00, 200.00]",160.0,200.00,180.000,"[205.00, 219.00]",205.0,219.00,212.000,"[210.00, 230.00]",210.0,230.00,220.000,"[220.00, 244.00]",220.0,244.0,232.00,"[225.00, 245.00]",225.0,245.0,235.0,"[230.00, 250.00]",230.0,250.0,240.00,"[240.00, 279.00]",240.0,279.0,259.5,"[245.00, 280.00]",245.0,280.0,262.5


In [13]:
# Set row as columns
ponoka_df.columns = ponoka_df.iloc[0]
# Normalize column names
ponoka_df.columns = [col.lower()\
    .replace(' ', '_')\
        .replace('_/_', '_')\
            .replace('-', 'to')\
                .replace('_|_', '_')\
                    .replace('$', 'cad')\
                        .replace(':', '')\
                            .replace('(', '')\
                                .replace(')', '') for col in ponoka_df.columns]
ponoka_df.head()

head_sold  \
Click in table to get category history. Week Ending:        Head Sold:   
Last Auction                            Wed Sep 14, 2022          2716   
                                        Wed Sep 14, 2022.1        2716   
Previous Auction 1                      Wed Sep 7, 2022           1476   
                                        Wed Sep 7, 2022.1         1476   

                                                            slaughter_cattle  \
Click in table to get category history. Week Ending:        Slaughter Cattle   
Last Auction                            Wed Sep 14, 2022    Slaughter Cattle   
                                        Wed Sep 14, 2022.1  Slaughter Cattle   
Previous Auction 1                      Wed Sep 7, 2022     Slaughter Cattle   
                                        Wed Sep 7, 2022.1   Slaughter Cattle   

                                                               d1_to_d2_cows  \
Click in table to get category history. Week Ending:            D1 - D2 Cows   
Last Auction                            Wed Sep 14, 2022    $95.00 - $115.00   
                                        Wed Sep 14, 2022.1  $95.00 - $115.00   
Previous Auction 1                      Wed Sep 7, 2022     $94.00 - $114.00   
                                        Wed Sep 7, 2022.1   $94.00 - $114.00   

                                                              d3_to_d4_cows  \
Click in table to get category history. Week Ending:           D3 - D4 Cows   
Last Auction                            Wed Sep 14, 2022    $80.00 - $94.00   
                                        Wed Sep 14, 2022.1  $80.00 - $94.00   
Previous Auction 1                      Wed Sep 7, 2022     $80.00 - $93.00   
                                        Wed Sep 7, 2022.1   $80.00 - $93.00   

                                                            good_holstein_cows  \
Click in table to get category history. Week Ending:        Good Holstein Cows   
Last Auction                            Wed Sep 14, 2022       $84.00 - $92.00   
                                        Wed Sep 14, 2022.1     $84.00 - $92.00   
Previous Auction 1                      Wed Sep 7, 2022        $83.00 - $88.00   
                                        Wed Sep 7, 2022.1      $83.00 - $88.00   

                                                            medium_holstein_cows  \
Click in table to get category history. Week Ending:        Medium Holstein Cows   
Last Auction                            Wed Sep 14, 2022         $72.00 - $83.00   
                                        Wed Sep 14, 2022.1       $72.00 - $83.00   
Previous Auction 1                      Wed Sep 7, 2022          $72.00 - $82.00   
                                        Wed Sep 7, 2022.1        $72.00 - $82.00   

                                                                  heiferettes  \
Click in table to get category history. Week Ending:              Heiferettes   
Last Auction                            Wed Sep 14, 2022    $120.00 - $155.00   
                                        Wed Sep 14, 2022.1  $120.00 - $155.00   
Previous Auction 1                      Wed Sep 7, 2022     $120.00 - $155.00   
                                        Wed Sep 7, 2022.1   $120.00 - $155.00   

                                                                bologna_bulls  \
Click in table to get category history. Week Ending:            Bologna Bulls   
Last Auction                            Wed Sep 14, 2022    $120.00 - $138.50   
                                        Wed Sep 14, 2022.1  $120.00 - $138.50   
Previous Auction 1                      Wed Sep 7, 2022     $110.00 - $138.50   
                                        Wed Sep 7, 2022.1   $110.00 - $138.50   

                                                                 feeder_bulls  \
Click in table to get category history. Week Ending:             Feeder Bulls   
Last Auction                            Wed S

In [14]:
ponoka_df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 7 entries, ('Click in table to get category history.', 'Week\xa0Ending:') to ('Previous Auction 2', 'Wed Aug 31, 2022.1')
Data columns (total 36 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   head_sold                 7 non-null      object
 1   slaughter_cattle          7 non-null      object
 2   d1_to_d2_cows             7 non-null      object
 3   d3_to_d4_cows             7 non-null      object
 4   good_holstein_cows        7 non-null      object
 5   medium_holstein_cows      7 non-null      object
 6   heiferettes               7 non-null      object
 7   bologna_bulls             7 non-null      object
 8   feeder_bulls              7 non-null      object
 9   replacement_cattle        7 non-null      object
 10  good_bred_cows            7 non-null      object
 11  older_bred_cows           7 non-null      object
 12  good_bred_heifers         7 non-nu

In [15]:
# Drop first row and check df
ponoka_df = ponoka_df.iloc[1:]
ponoka_df.info()
ponoka_df.head()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 6 entries, ('Last Auction', 'Wed Sep 14, 2022') to ('Previous Auction 2', 'Wed Aug 31, 2022.1')
Data columns (total 36 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   head_sold                 6 non-null      object
 1   slaughter_cattle          6 non-null      object
 2   d1_to_d2_cows             6 non-null      object
 3   d3_to_d4_cows             6 non-null      object
 4   good_holstein_cows        6 non-null      object
 5   medium_holstein_cows      6 non-null      object
 6   heiferettes               6 non-null      object
 7   bologna_bulls             6 non-null      object
 8   feeder_bulls              6 non-null      object
 9   replacement_cattle        6 non-null      object
 10  good_bred_cows            6 non-null      object
 11  older_bred_cows           6 non-null      object
 12  good_bred_heifers         6 non-null      object
 13  cow_ca

head_sold  slaughter_cattle  \
Last Auction       Wed Sep 14, 2022        2716  Slaughter Cattle   
                   Wed Sep 14, 2022.1      2716  Slaughter Cattle   
Previous Auction 1 Wed Sep 7, 2022         1476  Slaughter Cattle   
                   Wed Sep 7, 2022.1       1476  Slaughter Cattle   
Previous Auction 2 Wed Aug 31, 2022        3529  Slaughter Cattle   

                                           d1_to_d2_cows    d3_to_d4_cows  \
Last Auction       Wed Sep 14, 2022     $95.00 - $115.00  $80.00 - $94.00   
                   Wed Sep 14, 2022.1   $95.00 - $115.00  $80.00 - $94.00   
Previous Auction 1 Wed Sep 7, 2022      $94.00 - $114.00  $80.00 - $93.00   
                   Wed Sep 7, 2022.1    $94.00 - $114.00  $80.00 - $93.00   
Previous Auction 2 Wed Aug 31, 2022    $100.00 - $120.00  $85.00 - $99.00   

                                      good_holstein_cows medium_holstein_cows  \
Last Auction       Wed Sep 14, 2022      $84.00 - $92.00      $72.00 - $83.00   
                   Wed Sep 14, 2022.1    $84.00 - $92.00      $72.00 - $83.00   
Previous Auction 1 Wed Sep 7, 2022       $83.00 - $88.00      $72.00 - $82.00   
                   Wed Sep 7, 2022.1     $83.00 - $88.00      $72.00 - $82.00   
Previous Auction 2 Wed Aug 31, 2022      $88.00 - $94.00      $74.00 - $87.00   

                                             heiferettes      bologna_bulls  \
Last Auction       Wed Sep 14, 2022    $120.00 - $155.00  $120.00 - $138.50   
                   Wed Sep 14, 2022.1  $120.00 - $155.00  $120.00 - $138.50   
Previous Auction 1 Wed Sep 7, 2022     $120.00 - $155.00  $110.00 - $138.50   
                   Wed Sep 7, 2022.1   $120.00 - $155.00  $110.00 - $138.50   
Previous Auction 2 Wed Aug 31, 2022    $120.00 - $155.00  $115.00 - $155.00   

                                            feeder_bulls  replacement_cattle  \
Last Auction       Wed Sep 14, 2022    $140.00 - $155.00  Replacement Cattle   
                   Wed Sep 14, 2022.1  $140.00 - $155.00  Replacement Cattle   
Previous Auction 1 Wed Sep 7, 2022     $120.00 - $150.00  Replacement Cattle   
                   Wed Sep 7, 2022.1   $120.00 - $150.00  Replacement Cattle   
Previous Auction 2 Wed Aug 31, 2022    $125.00 - $160.00  Replacement Cattle   

                                      good_bred_cows older_bred_cows  \
Last Auction       Wed Sep 14, 2022            — - —           — - —   
                   Wed Sep 14, 2022.1          — - —           — - —   
Previous Auction 1 Wed Sep 7, 2022             — - —           — - —   
                   Wed Sep 7, 2022.1           — - —           — - —   
Previous Auction 2 Wed Aug 31, 2022            — - —           — - —   

                                      good_bred_heifers  \
Last Auction       Wed Sep 14, 2022               — - —   
                   Wed Sep 14, 2022.1             — - —   
Previous Auction 1 Wed Sep 7, 2022                — - —   
                   Wed Sep 7, 2022.1              — - —   
Previous Auction 2 Wed Aug 31, 2022               — - —   

                                      cow_calf_pairs_younger  \
Last Auction       Wed Sep 14, 2022                    — - —   
                   Wed Sep 14, 2022.1                  — - —   
Previous Auction 1 Wed Sep 7, 2022                     — - —   
                   Wed Sep 7, 2022.1                   — - —   
Previous Auction 2 Wed Aug 31, 2022                    — - —   

                                      cow_calf_pairs_older  \
Last Auction       Wed Sep 14, 2022                  — - —   
                   Wed Sep 14, 2022.1                — - —   
Previous Auction 1 Wed Sep 7, 2022                   — - —   
                   Wed Sep 7, 2022.1                 — - —   
Previous Auction 2 Wed Aug 31, 2022                  — - —   

                                       stock_and_feeder_cattle   weight  \
Last Auction       Wed Sep 14, 2022    Stock and Feeder Cattle   Steers   
                   We

In [16]:
print(ponoka_df.index)
type(ponoka_df.index)

MultiIndex([(      'Last Auction',   'Wed Sep 14, 2022'),
            (      'Last Auction', 'Wed Sep 14, 2022.1'),
            ('Previous Auction 1',    'Wed Sep 7, 2022'),
            ('Previous Auction 1',  'Wed Sep 7, 2022.1'),
            ('Previous Auction 2',   'Wed Aug 31, 2022'),
            ('Previous Auction 2', 'Wed Aug 31, 2022.1')],
           )


pandas.core.indexes.multi.MultiIndex

In [17]:
dt_idx = ponoka_df.index.get_level_values(1)
ponoka_df = ponoka_df.set_index(dt_idx)
ponoka_df

,head_sold,slaughter_cattle,d1_to_d2_cows,d3_to_d4_cows,good_holstein_cows,medium_holstein_cows,heiferettes,bologna_bulls,feeder_bulls,replacement_cattle,good_bred_cows,older_bred_cows,good_bred_heifers,cow_calf_pairs_younger,cow_calf_pairs_older,stock_and_feeder_cattle,weight,1000_lbs_steers_heifers,900_lbs_steers_heifers,800_lbs_steers_heifers,700_lbs_steers_heifers,600_lbs_steers_heifers,500_lbs_steers_heifers,400_lbs_steers_heifers,300_lbs_steers_heifers,dairy_&_calves,dairy_steers,baby_calves_dairy_cad/hd,baby_calves_beef_cad/hd,feed,hay_square_bales,hay_round_bales,straw_square_bales,straw_round_bales,greenfeed_square_bales,greenfeed_round_bales
"Wed Sep 14, 2022",2716,Slaughter Cattle,$95.00 - $115.00,$80.00 - $94.00,$84.00 - $92.00,$72.00 - $83.00,$120.00 - $155.00,$120.00 - $138.50,$140.00 - $155.00,Replacement Cattle,— - —,— - —,— - —,— - —,— - —,Stock and Feeder Cattle,Steers,$210.00 - $222.75,$220.00 - $232.25,$230.00 - $247.00,$240.00 - $260.50,$250.00 - $278.00,$275.00 - $307.00,$275.00 - $305.00,$250.00 - $310.00,Dairy & Calves,$135.00 - $165.00,— - —,— - —,Feed,— - $5.00,— - $120.00,— - —,— - —,— - —,— - —
"Wed Sep 14, 2022.1",2716,Slaughter Cattle,$95.00 - $115.00,$80.00 - $94.00,$84.00 - $92.00,$72.00 - $83.00,$120.00 - $155.00,$120.00 - $138.50,$140.00 - $155.00,Replacement Cattle,— - —,— - —,— - —,— - —,— - —,Stock and Feeder Cattle,Heifers,$160.00 - $200.00,$205.00 - $219.00,$210.00 - $230.00,$220.00 - $244.00,$225.00 - $245.00,$230.00 - $250.00,$240.00 - $279.00,$245.00 - $280.00,Dairy & Calves,$135.00 - $165.00,— - —,— - —,Feed,— - $5.00,— - $120.00,— - —,— - —,— - —,— - —
"Wed Sep 7, 2022",1476,Slaughter Cattle,$94.00 - $114.00,$80.00 - $93.00,$83.00 - $88.00,$72.00 - $82.00,$120.00 - $155.00,$110.00 - $138.50,$120.00 - $150.00,Replacement Cattle,— - —,— - —,— - —,— - —,— - —,Stock and Feeder Cattle,Steers,$214.00 - $220.00,$210.00 - $236.50,$235.00 - $253.00,$235.00 - $257.00,$240.00 - $267.00,$245.00 - $276.00,$250.00 - $284.00,$250.00 - $305.00,Dairy & Calves,$120.00 - $160.00,— - —,— - —,Feed,— - —,— - —,— - $2.50,— - —,— - —,— - —
"Wed Sep 7, 2022.1",1476,Slaughter Cattle,$94.00 - $114.00,$80.00 - $93.00,$83.00 - $88.00,$72.00 - $82.00,$120.00 - $155.00,$110.00 - $138.50,$120.00 - $150.00,Replacement Cattle,— - —,— - —,— - —,— - —,— - —,Stock and Feeder Cattle,Heifers,$165.00 - $190.00,$200.00 - $218.00,$215.00 - $236.00,$225.00 - $247.00,$225.00 - $247.00,$195.00 - $230.00,$200.00 - $245.00,$215.00 - $250.00,Dairy & Calves,$120.00 - $160.00,— - —,— - —,Feed,— - —,— - —,— - $2.50,— - —,— - —,— - —
"Wed Aug 31, 2022",3529,Slaughter Cattle,$100.00 - $120.00,$85.00 - $99.00,$88.00 - $94.00,$74.00 - $87.00,$120.00 - $155.00,$115.00 - $155.00,$125.00 - $160.00,Replacement Cattle,— - —,— - —,— - —,— - —,— - —,Stock and Feeder Cattle,Steers,$215.00 - $226.75,$225.00 - $241.50,$230.00 - $253.00,$240.00 - $264.50,$245.00 - $265.50,$245.00 - $270.00,$230.00 - $288.00,$230.00 - $282.50,Dairy & Calves,$125.00 - $155.00,— - —,— - —,Feed,— - —,— - —,— - —,— - —,— - —,— - —
"Wed Aug 31, 2022.1",3529,Slaughter Cattle,$100.00 - $120.00,$85.00 - $99.00,$88.00 - $94.00,$74.00 - $87.00,$120.00 - $155.00,$115.00 - $155.00,$125.00 - $160.00,Replacement Cattle,— - —,— - —,— - —,— - —,— - —,Stock and Feeder Cattle,Heifers,$170.00 - $203.00,$195.00 - $219.50,$210.00 - $234.50,$225.00 - $248.00,$230.00 - $259.50,$200.00 - $236.00,$205.00 - $240.00,$210.00 - $240.00,Dairy & Calves,$125.00 - $155.00,— - —,— - —,Feed,— - —,— - —,— - —,— - —,— - —,— - —


In [21]:
ponoka_df.drop(['Wed Sep 14, 2022.1', 'Wed Sep 7, 2022.1', 'Wed Aug 31, 2022.1'])

rgx = r'.\.1'
rows_to_drop = []


In [24]:
print(ponoka_df.index.str.contains(rgx))


[False  True False  True False  True]


In [25]:
# Dropping duplicate rows from df
ponoka_df = ponoka_df[ponoka_df.index.str.contains(rgx) == False]
ponoka_df

,head_sold,slaughter_cattle,d1_to_d2_cows,d3_to_d4_cows,good_holstein_cows,medium_holstein_cows,heiferettes,bologna_bulls,feeder_bulls,replacement_cattle,good_bred_cows,older_bred_cows,good_bred_heifers,cow_calf_pairs_younger,cow_calf_pairs_older,stock_and_feeder_cattle,weight,1000_lbs_steers_heifers,900_lbs_steers_heifers,800_lbs_steers_heifers,700_lbs_steers_heifers,600_lbs_steers_heifers,500_lbs_steers_heifers,400_lbs_steers_heifers,300_lbs_steers_heifers,dairy_&_calves,dairy_steers,baby_calves_dairy_cad/hd,baby_calves_beef_cad/hd,feed,hay_square_bales,hay_round_bales,straw_square_bales,straw_round_bales,greenfeed_square_bales,greenfeed_round_bales
"Wed Sep 14, 2022",2716,Slaughter Cattle,$95.00 - $115.00,$80.00 - $94.00,$84.00 - $92.00,$72.00 - $83.00,$120.00 - $155.00,$120.00 - $138.50,$140.00 - $155.00,Replacement Cattle,— - —,— - —,— - —,— - —,— - —,Stock and Feeder Cattle,Steers,$210.00 - $222.75,$220.00 - $232.25,$230.00 - $247.00,$240.00 - $260.50,$250.00 - $278.00,$275.00 - $307.00,$275.00 - $305.00,$250.00 - $310.00,Dairy & Calves,$135.00 - $165.00,— - —,— - —,Feed,— - $5.00,— - $120.00,— - —,— - —,— - —,— - —
"Wed Sep 7, 2022",1476,Slaughter Cattle,$94.00 - $114.00,$80.00 - $93.00,$83.00 - $88.00,$72.00 - $82.00,$120.00 - $155.00,$110.00 - $138.50,$120.00 - $150.00,Replacement Cattle,— - —,— - —,— - —,— - —,— - —,Stock and Feeder Cattle,Steers,$214.00 - $220.00,$210.00 - $236.50,$235.00 - $253.00,$235.00 - $257.00,$240.00 - $267.00,$245.00 - $276.00,$250.00 - $284.00,$250.00 - $305.00,Dairy & Calves,$120.00 - $160.00,— - —,— - —,Feed,— - —,— - —,— - $2.50,— - —,— - —,— - —
"Wed Aug 31, 2022",3529,Slaughter Cattle,$100.00 - $120.00,$85.00 - $99.00,$88.00 - $94.00,$74.00 - $87.00,$120.00 - $155.00,$115.00 - $155.00,$125.00 - $160.00,Replacement Cattle,— - —,— - —,— - —,— - —,— - —,Stock and Feeder Cattle,Steers,$215.00 - $226.75,$225.00 - $241.50,$230.00 - $253.00,$240.00 - $264.50,$245.00 - $265.50,$245.00 - $270.00,$230.00 - $288.00,$230.00 - $282.50,Dairy & Calves,$125.00 - $155.00,— - —,— - —,Feed,— - —,— - —,— - —,— - —,— - —,— - —


In [35]:
# Columns of interest: Steers & Heifers 300-1000 lbs
ponoka_steers = ponoka_df.copy()[[
    'weight',
    '1000_lbs_steers_heifers', 
    '900_lbs_steers_heifers',
    '800_lbs_steers_heifers',
    '700_lbs_steers_heifers',
    '600_lbs_steers_heifers',
    '500_lbs_steers_heifers',
    '400_lbs_steers_heifers',
    '300_lbs_steers_heifers']]
    
ponoka_steers



,weight,1000_lbs_steers_heifers,900_lbs_steers_heifers,800_lbs_steers_heifers,700_lbs_steers_heifers,600_lbs_steers_heifers,500_lbs_steers_heifers,400_lbs_steers_heifers,300_lbs_steers_heifers
"Wed Sep 14, 2022",Steers,$210.00 - $222.75,$220.00 - $232.25,$230.00 - $247.00,$240.00 - $260.50,$250.00 - $278.00,$275.00 - $307.00,$275.00 - $305.00,$250.00 - $310.00
"Wed Sep 7, 2022",Steers,$214.00 - $220.00,$210.00 - $236.50,$235.00 - $253.00,$235.00 - $257.00,$240.00 - $267.00,$245.00 - $276.00,$250.00 - $284.00,$250.00 - $305.00
"Wed Aug 31, 2022",Steers,$215.00 - $226.75,$225.00 - $241.50,$230.00 - $253.00,$240.00 - $264.50,$245.00 - $265.50,$245.00 - $270.00,$230.00 - $288.00,$230.00 - $282.50
